In [4]:
#IMPORT STATEMENTS
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize
import os

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Harshita\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Metadata Extraction 

### Structure of Metadata 

In [37]:
# Load metadata schema CSV
schema_file = r"C:\Users\Harshita\Downloads\Metadata Scheme ADG.csv"
metadata_schema = pd.read_csv(schema_file,delimiter="\t" )
metadata_schema

,Standort,Archiv ID,PROBANDNR,DOK_ART,ARCHIVORT,PROVENIENZ,SPERRUNG,ENTSTZEIT,Zeitumfang 1,NAME,...,PART_KONFESSION,PART_HERKU,PART_SCHUL,PART_AUSBI,PART_STAND,PART_BERUF,PART_POLOR,PART_PKONV,PART_ENGAG,KRIT10
0,IGNORE,IGNORE,IGNORE,IGNORE,IGNORE,IGNORE,IGNORE,IGNORE,IGNORE,IGNORE,...,confession life partner,place of birth life partner,school graduation life partner,apprenticeship/training life partner,social status life partner,profession life partner,political attitude life partner,change of political attitude life partner,social commitment life partner,IGNORE


In [38]:
import pandas as pd
import json

def convert_dataframe_to_json(df, output_json):
    """
    Converts a DataFrame into a structured JSON format.

    Args:
        df (pd.DataFrame): The DataFrame containing the metadata schema.
        output_json (str): Path to save the output JSON file.

    Returns:
        str: A JSON string representation of the DataFrame.
    """
    try:
        # Convert DataFrame to JSON
        json_data = df.to_json(orient="records", indent=4)

        # Save JSON to file
        with open(output_json, "w", encoding="utf-8") as f:
            f.write(json_data)

        print("\n✅ DataFrame Successfully Converted to JSON!")
        return json_data  # Return JSON string for verification

    except Exception as e:
        print(f"❌ Error converting DataFrame to JSON: {e}")
        return "{}"  # Return empty JSON if error occurs

# Example usage
output_json = r"C:\Users\Harshita\Downloads\case study\metadata_schema.json"

# Convert DataFrame to JSON
metadata_schema_json = convert_dataframe_to_json(metadata_schema, output_json)

# Print JSON output
print("\n✅ Converted JSON Data:")
print(metadata_schema_json)



✅ DataFrame Successfully Converted to JSON!

✅ Converted JSON Data:
[
    {
        "Standort":"IGNORE",
        "Archiv ID":"IGNORE",
        "PROBANDNR":"IGNORE",
        "DOK_ART":"IGNORE",
        "ARCHIVORT":"IGNORE",
        "PROVENIENZ":"IGNORE",
        "SPERRUNG":"IGNORE",
        "ENTSTZEIT":"IGNORE",
        "Zeitumfang 1":"IGNORE",
        "NAME":"IGNORE",
        "VORNAME":"IGNORE",
        "ORT":"place of living",
        "Feld1":"IGNORE",
        "PSEUDONYM":"IGNORE",
        "GESCHLECHT":"GENDER",
        "JAHRGANG":"year of birth",
        "IPV":"IGNORE",
        "DATENBOGEN":"IGNORE",
        "KURZBESCHR":"IGNORE",
        "TITEL":"IGNORE",
        "STRASSE":"IGNORE",
        "PLZ":"IGNORE",
        "TELEFON":"IGNORE",
        "GRUPPE":"IGNORE",
        "BERUF":"profession",
        "HEUT_FAMST":"recent civil status",
        "INTERVIEWE":"IGNORE",
        "TIPPER":"IGNORE",
        "Segmentierung":"IGNORE",
        "DATUM1":"IGNORE",
        "DATUM2":"IGNORE",
     

### The Metadata has 136 columns to be filled 

### Loading the transcipts 

In [5]:
def load_interview_csvs(folder_path):
    """
    Loads all interview CSV files from the specified folder.

    Args:
        folder_path (str): Path to the directory containing interview CSV files.

    Returns:
        dict: A dictionary where keys are filenames and values are DataFrames.
    """
    interview_data = {}

    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        
        # Check if it's a valid CSV file
        if os.path.isfile(file_path) and filename.endswith(".csv"):
            print(f"\n📂 Loading interview file: {filename}")
            try:
                input_data = pd.read_csv(file_path, sep=None, engine='python')
                interview_data[filename] = input_data
            except Exception as e:
                print(f"❌ Error loading {filename}: {e}")

    return interview_data

# # Example usage:
# folder_path = r"C:\Users\Harshita\Downloads\case study\Transcripts"
# interview_csvs = load_interview_csvs(folder_path)



### Transcript Processing Functions

In [6]:
def chunk_transcript(data: pd.DataFrame) -> pd.DataFrame:
    
    # Ensure 'Transkript' column has no NaN values
    data['Transkript'] = data['Transkript'].fillna("").astype(str)

    # Chunking logic
    chunked_data = []
    current_speaker = None
    current_text = ""
    initial_timestamp = ""

    for index, row in data.iterrows():
        speaker = row['Sprecher']
        transcript = row['Transkript']
        timestamp = row['Timecode'] 

        if speaker != current_speaker:
            # Save previous chunk if exists
            if current_speaker is not None:
                chunked_data.append({
                    'Speaker': current_speaker,
                    'Transcript': current_text.strip(),
                    'Initial_Timestamp' : initial_timestamp,
                    'Current_Timestamp' : timestamp
                })
            
            # Start a new chunk
            current_speaker = speaker
            current_text = transcript
            initial_timestamp = timestamp 
        else:
            # Continue appending to the same speaker's chunk
            current_text += " " + transcript if isinstance(transcript, str) else ""

    # Save the last chunk
    if current_speaker is not None:
        chunked_data.append({
            'Speaker': current_speaker,
            'Transcript': current_text.strip(),
            'Initial_Timestamp' : initial_timestamp,
            'Current_Timestamp' : timestamp
        })

    # Convert to DataFrame
    chunked_df = pd.DataFrame(chunked_data)
    
    return chunked_df

In [7]:
def chunk_by_sentence(chunked_df: pd.DataFrame, min_tokens=256, max_tokens=512) -> pd.DataFrame:
    """
    Further splits chunks by sentence while ensuring each chunk is within a token range.

    Args:
        chunked_df (pd.DataFrame): Input DataFrame with 'Speaker' and 'Transcript' columns.
        min_tokens (int): Minimum number of tokens per chunk.
        max_tokens (int): Maximum number of tokens per chunk.

    Returns:
        pd.DataFrame: DataFrame with sentence-based chunked transcripts.
    """

    # Function to count tokens (approximate, assuming 1 word ≈ 1.2 tokens)
    def count_tokens(text):
        return len(text.split()) * 1.2  # Rough estimate

    # Initialize list for final merged chunks
    merged_chunks = []
    temp_chunk = []
    temp_token_count = 0
    speaker = None
    initial_timestamp = ""
    final_timestamp = ""

    # Process each row
    for _, row in chunked_df.iterrows():
        sentence = row['Transcript']
        sentence_tokens = count_tokens(sentence)

        # If adding this chunk keeps us within MAX_TOKENS
        if temp_token_count + sentence_tokens <= max_tokens:
            if not temp_chunk:
                speaker = row['Speaker']  # Store speaker only for new chunks
                initial_timestamp = row['Initial_Timestamp']
            temp_chunk.append(sentence)
            temp_token_count += sentence_tokens
        else:
            # Save the previous chunk before starting a new one
            if temp_chunk:
                merged_chunks.append({
                    'Speaker': speaker,
                    'Transcript': " ".join(temp_chunk),
                    'Initial_Timestamp' : initial_timestamp,
                    'Current_Timestamp' : row['Current_Timestamp']
                })

            # Start a new chunk with the current sentence
            temp_chunk = [sentence]
            temp_token_count = sentence_tokens
            speaker = row['Speaker']
            initial_timestamp = row['Initial_Timestamp']
            final_timestamp = row['Current_Timestamp']

    # Save last chunk if any content remains
    if temp_chunk:
        merged_chunks.append({
            'Speaker': speaker,
            'Transcript': " ".join(temp_chunk),
            'Initial_Timestamp' : initial_timestamp,
            'Current_Timestamp' : final_timestamp
        })

    # Convert to DataFrame
    final_merged_df = pd.DataFrame(merged_chunks)
    
    return final_merged_df

### Using Groq

In [77]:
from groq import Groq
import pandas as pd 


# api_key = "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"
# api_key = "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"
api_key = "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"
# api_key = "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"
# api_key = "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"
# api_key = "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"
#Initialize the Groq client
client = Groq(api_key=api_key)

print("Groq client initialized successfully!")

Groq client initialized successfully!


In [67]:
import pandas as pd
import json
import re

def extract_metadata(client, model_name: str, chunks: pd.DataFrame, metadata_schema: dict) -> pd.DataFrame:
    """
    Extract metadata from multiple chunks of a German transcript using the specified model.

    Args:
        client: The Groq client object.
        model_name: The AI model to use.
        chunks: A DataFrame with 'Speaker' and 'Transcript' columns.
        metadata_schema: A dictionary representing the metadata schema.

    Returns:
        A DataFrame containing structured extracted metadata.
    """
    all_metadata = []

    for i, row in chunks.iterrows():
        speaker = row["Speaker"]
        transcript = row["Transcript"][:1500]  # Prevent AI truncation
        timestamp = row["Timestamp"]

        print(f"Processing chunk {i+1}/{len(chunks)} for Speaker: {speaker}...")

        # Dynamically structure the JSON schema for the prompt
        schema_json = json.dumps(metadata_schema, indent=2)

        # JSON-structured prompt
        prompt = f"""
        You are a highly skilled metadata extraction AI. Your task is to analyze the provided transcript 
        and extract structured metadata according to a predefined schema. The metadata schema contains field names and their descriptions.

        **Your Task:**
        1. Extract relevant metadata from the transcript while adhering **strictly** to the JSON schema.
        2. Fields labeled `"IGNORE"` should **not be discarded**—instead, use the column name as its description.
        3. If information is **missing** in the transcript,  set it to `"???"`.

        **📌 Metadata Extraction Schema**
        Extract metadata using the **following JSON schema**:

        
        {schema_json}
       

        **📌 Output Format**
        Your response **must** be a **valid JSON object** following the schema, with no extra text.

        ---
        **📌 Transcript:**
        {transcript}
        """

        # Generate response using the Groq client
        response = client.chat.completions.create(
            model=model_name,
            messages=[{"role": "user", "content": prompt}],
            temperature=0,
        )

        # Extract raw response
        response_text = response.choices[0].message.content.strip()

        # Debugging: Print the raw AI response to identify issues
        print(f"\n🔍 DEBUG: Raw AI Response for chunk {i+1}:\n{response_text}\n")

        # **Fix Hidden JSON Formatting Issues**
        response_text = response_text.strip("`")  # Remove possible markdown backticks
        response_text = re.sub(r'```json|```', '', response_text).strip()  # Remove ```json blocks

        # Parse AI response to JSON
        try:
            metadata = json.loads(response_text)  # Convert response to JSON
        except json.JSONDecodeError as e:
            print(f"❌ JSON Error in chunk {i+1}: {e}\nAI Response:\n{response_text}\nSkipping this chunk...")
            continue

        # Clean metadata: remove invalid values
        extracted_metadata = {key: value if value not in ["%%%", None, "", "null"] else None for key, value in metadata.items()}
        extracted_metadata["Speaker"] = speaker
        extracted_metadata["Timestamp"] = timestamp

        all_metadata.append(extracted_metadata)

    # Convert to DataFrame
    return pd.DataFrame(all_metadata)


In [75]:
import pandas as pd
import json
import re

def extract_metadate(client, model_name: str, chunks: pd.DataFrame, metadata_schema: dict) -> pd.DataFrame:
    """
    Extract metadata from multiple chunks of a transcript using the specified model.

    Args:
        client: The AI client object.
        model_name: The AI model to use.
        chunks: A DataFrame with 'Speaker' and 'Transcript' columns.
        metadata_schema: A dictionary representing the metadata schema.

    Returns:
        pd.DataFrame: A structured DataFrame containing extracted metadata.
    """
    all_metadata = []

    for i, row in chunks.iterrows():
        speaker = row["Speaker"]
        transcript = row["Transcript"][:1500]  # Prevent AI truncation
        timestamp = row["Timestamp"]

        # Convert schema to JSON format
        schema_json = json.dumps(metadata_schema, indent=2)

        
          # JSON-structured prompt
        prompt = f"""
        Extrahieren Sie die folgenden Informationen aus dem Transkript und geben Sie die Antwort als **valide JSON-Struktur** zurück.
        Falls eine Information nicht gefunden wird, setzen Sie den Wert als `null`.
        Antwort **nur als JSON**, keine Erklärungen oder zusätzlichen Texte.

        Geben Sie die Antwort exakt in folgendem Format:
        {schema_json}

        Antwort nur als JSON:

        Transkript:
        {transcript}
        """

        # Generate response using AI model
        response = client.chat.completions.create(
            model=model_name,
            messages=[{"role": "user", "content": prompt}],
            temperature=0,
        )


        # Extract raw response
        response_text = response.choices[0].message.content.strip()

        # Debugging: Print the raw AI response to identify issues
        print(f"\n🔍 DEBUG: Raw AI Response for chunk {i+1}:\n{response_text}\n")

        # **Fix Hidden JSON Formatting Issues**
        response_text = response_text.strip("`")  # Remove possible markdown backticks
        response_text = re.sub(r'```json|```', '', response_text).strip()  # Remove ```json blocks

        # Parse AI response to JSON
        try:
            metadata = json.loads(response_text)  # Convert response to JSON
            if isinstance(metadata, list):  # If AI returned a list, take the first object
                metadata = metadata[0]
        except json.JSONDecodeError as e:
            print(f"❌ JSON Error in chunk {i+1}: {e}\nAI Response:\n{response_text}\nSkipping this chunk...")
            metadata = {key: "???" for key in metadata_schema.keys()}  # Fill with `"???"`

        # Clean metadata: remove invalid values
        extracted_metadata = {key: value if value not in ["%%%", None, "", "null"] else None for key, value in metadata.items()}
        extracted_metadata["Speaker"] = speaker
        extracted_metadata["Timestamp"] = timestamp

        all_metadata.append(extracted_metadata)

    # Convert to DataFrame
    return pd.DataFrame(all_metadata)


In [78]:
import os
import time
import pandas as pd
import json

# Define Paths
folder_path = r"C:\Users\Harshita\Downloads\case study\Transcripts"
schema_file = r"C:\Users\Harshita\Downloads\case study\metadata_schema.json"
MODEL = "llama-3.3-70b-versatile"

# Load Metadata Schema from JSON
with open(schema_file, "r", encoding="utf-8") as f:
    metadata_schema = json.load(f)

# List to store metadata for all files
all_metadata = []

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    
    if os.path.isfile(file_path) and filename.endswith(".csv"):
        print(f"\nProcessing file: {filename}")

        # Load transcript CSV
        input_data = pd.read_csv(file_path, sep=None, engine='python')

        # Process transcript into chunks
        speaker_chunks_df = chunk_transcript(input_data)
        final_chunks_df = chunk_by_sentence(speaker_chunks_df)

        # Create timestamp column
        final_chunks_df["Timestamp"] = final_chunks_df["Initial_Timestamp"] + " - " + final_chunks_df["Current_Timestamp"]
        final_chunks_df.drop(columns=["Initial_Timestamp", "Current_Timestamp"], inplace=True)

        # Extract metadata for the chunks
        llama_70b_responses = extract_metadate(client, MODEL, final_chunks_df, metadata_schema)

        if not llama_70b_responses.empty:
            # Merge chunk outputs into a single row
            merged_metadata = llama_70b_responses.apply(lambda col: ' | '.join(col.dropna().astype(str)))

            for column in merged_metadata.index:
                unique_values = set(value.strip() for value in merged_metadata[column].strip().split(","))
                merged_metadata[column] = " | ".join(filter(None, unique_values))

            # Append to list
            all_metadata.append(merged_metadata)
        else:
            print(f"No metadata extracted from {filename}")

        time.sleep(0.5)

# Convert list of metadata rows into a single DataFrame
final_metadata_df = pd.DataFrame(all_metadata)




Processing file: adg0001_er_2024_10_31.csv

🔍 DEBUG: Raw AI Response for chunk 1:
[
  {
    "Standort": null,
    "Archiv ID": null,
    "PROBANDNR": null,
    "DOK_ART": null,
    "ARCHIVORT": null,
    "PROVENIENZ": null,
    "SPERRUNG": null,
    "ENTSTZEIT": null,
    "Zeitumfang 1": null,
    "NAME": null,
    "VORNAME": null,
    "ORT": "Hemer im Sauerland",
    "Feld1": null,
    "PSEUDONYM": null,
    "GESCHLECHT": null,
    "JAHRGANG": "1925",
    "IPV": null,
    "DATENBOGEN": null,
    "KURZBESCHR": null,
    "TITEL": null,
    "STRASSE": null,
    "PLZ": null,
    "TELEFON": null,
    "GRUPPE": null,
    "BERUF": null,
    "HEUT_FAMST": null,
    "INTERVIEWE": null,
    "TIPPER": null,
    "Segmentierung": null,
    "DATUM1": null,
    "DATUM2": null,
    "DATUM3": null,
    "DAUER": null,
    "online": null,
    "AUSDRUCKSART": null,
    "UNKAUSDRUC": null,
    "KORRAUSDRU": null,
    "SCHLAGWORT": null,
    "KURZBIOGRA": null,
    "KURZPROTOK": null,
    "FOTOS": null,
 

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01jkng267af8aa3gy2hazjmd8h` service tier `on_demand` on : Limit 100000, Used 99177, Requested 1822. Please try again in 14m22.896s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': '', 'code': 'rate_limit_exceeded'}}

In [52]:
import pandas as pd
import json
import re

def extract_metadata(client, model_name: str, chunks: pd.DataFrame, metadata_schema: dict) -> pd.DataFrame:
    """
    Extract metadata from multiple chunks of a transcript using the specified model.

    Args:
        client: The AI client object.
        model_name: The AI model to use.
        chunks: A DataFrame with 'Speaker' and 'Transcript' columns.
        metadata_schema: A dictionary representing the metadata schema.

    Returns:
        pd.DataFrame: A structured DataFrame containing extracted metadata.
    """
    all_metadata = []

    for i, row in chunks.iterrows():
        speaker = row["Speaker"]
        transcript = row["Transcript"][:1500]  # Prevent AI truncation
        timestamp = row["Timestamp"]

        # Convert schema to JSON format
        schema_json = json.dumps(metadata_schema, indent=2)

        # JSON-structured prompt
        prompt = f"""
       
        You are a highly skilled metadata extraction AI. Your task is to analyze the provided transcript 
        and extract structured metadata according to a predefined schema. The metadata schema contains field names and their descriptions.

        **Your Task:**
        1. Extract relevant metadata from the transcript while adhering **strictly** to the JSON schema.
        2. Fields labeled `"IGNORE"` should **not be discarded**—instead, use the column name as its description.
        3. If information is **missing** in the transcript, infer a reasonable value or set it to `"null"`.
        4. Ensure all required fields are included and that the extracted metadata is in **valid JSON format**.

        ** Metadata Extraction Schema**
        Extract metadata using the **following JSON schema**:

        ```json
        {schema_json}
        ```

        Transcript:
        {transcript}
        """

        # Generate response using AI model
        response = client.chat.completions.create(
            model=model_name,
            messages=[{"role": "user", "content": prompt}],
            temperature=0,
        )

        # Extract raw response
        response_text = response.choices[0].message.content.strip()

        # Fix Hidden JSON Formatting Issues
        response_text = response_text.strip("`")  # Remove possible markdown backticks
        response_text = re.sub(r'```json|```', '', response_text).strip()  # Remove ```json blocks

        # Parse AI response to JSON
        try:
            metadata = json.loads(response_text)  # Convert response to JSON
        except json.JSONDecodeError:
            continue  # Skip this chunk if JSON parsing fails

        # Clean metadata: remove invalid values
        extracted_metadata = {key: value if value not in ["%%%", None, "", "null"] else None for key, value in metadata.items()}
        extracted_metadata["Speaker"] = speaker
        extracted_metadata["Timestamp"] = timestamp

        all_metadata.append(extracted_metadata)

    # Convert to DataFrame
    return pd.DataFrame(all_metadata)
